## Imports

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
import itertools
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
import cv2
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint, EarlyStopping
warnings.simplefilter(action='ignore', category=FutureWarning)

## Train and Test image directory paths

In [3]:
train_path = r'D:\gesture\train'
test_path = r'D:\gesture\test'

In [4]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)

Found 629 images belonging to 3 classes.
Found 413 images belonging to 3 classes.


## Building the model

In [42]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Flatten())

model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(3,activation ="softmax"))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

model.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0005)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

history2 = model.fit(train_batches, epochs=100, callbacks=[reduce_lr, early_stop],  validation_data = test_batches)

Epoch 1/100
63/63 [==============================] - 6s 54ms/step - loss: 1.7751 - accuracy: 0.8394 - val_loss: 0.4647 - val_accuracy: 0.7918
Epoch 2/100
63/63 [==============================] - 3s 40ms/step - loss: 0.0568 - accuracy: 0.9889 - val_loss: 0.3920 - val_accuracy: 0.8571
Epoch 3/100
63/63 [==============================] - 3s 40ms/step - loss: 0.0169 - accuracy: 0.9984 - val_loss: 0.3041 - val_accuracy: 0.9153
Epoch 4/100
63/63 [==============================] - 3s 41ms/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.3211 - val_accuracy: 0.9080
Epoch 5/100
63/63 [==============================] - 3s 48ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.3173 - val_accuracy: 0.9104


In [43]:
history2.history

{'loss': [1.775146484375,
  0.0567680187523365,
  0.016947966068983078,
  0.00819460116326809,
  0.0055899121798574924],
 'accuracy': [0.839427649974823,
  0.9888712167739868,
  0.998410165309906,
  1.0,
  1.0],
 'val_loss': [0.46471813321113586,
  0.39198121428489685,
  0.30411002039909363,
  0.32105693221092224,
  0.31727659702301025],
 'val_accuracy': [0.7917675375938416,
  0.8571428656578064,
  0.9152542352676392,
  0.9079903364181519,
  0.9104115962982178],
 'lr': [0.001, 0.001, 0.001, 0.001, 0.0005]}

## Saving our model

In [44]:
model.save('sign_model.h5')

## Metrics

In [45]:
imgs, labels = next(test_batches) 
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))

loss of 0.0189112089574337; accuracy of 100.0%


In [49]:
imgs, labels = next(test_batches)
model = keras.models.load_model(r"sign_model.h5")
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))

loss of 0.3421238958835602; accuracy of 89.99999761581421%


In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 31, 31, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

## Predictions

In [48]:
imgs, labels = next(test_batches) 

word_dict = {0:'One', 1:'Two', 2:'Three'}

predictions = model.predict(imgs, verbose=0)

print("Predicted labels")
for ind, i in enumerate(predictions):
    print(word_dict[np.argmax(i)])
    
print('Actual labels')
for i in labels:
    print(word_dict[np.argmax(i)])

Predicted labels
Two
Two
One
Two
Three
Three
Three
Three
One
One
Actual labels
Two
Two
One
Two
Three
Three
Three
Two
One
One
